# Setup

We recommend using [Google Collab](https://colab.google/), [Kaggle](https://www.kaggle.com/), or a similar hosting environment. Once you have this notebook running in an environment. Use the below code block to install the following libraries: `paramiko, rigging, marque`. If you need help installing the environment or getting setup, let us know. When you're finished installing the requirements, make sure you restart the kernel. "
   

In [ ]:
!pip install -U paramiko rigging marque

# Rigging

We'll do some excercises to get familiar with our [rigging](https://github.com/dreadnode/rigging) library for working with LLMs. (We )

We can connect to our VLLM server using the "connection string" style mechanism.

In [ ]:
# Exercise 1
#
# Use rigging to collect the response for `Say Hello!`
#

import rigging as rg

connection_string = 'mistral/mistral-large-latest,api_key=<your_api_key>'

generator = rg.get_generator(connection_string)

chat = generator.chat([{'role': 'user', 'content': 'Say Hello'}]).run()
# ...

In [ ]:
chat.last

In [ ]:
# Exercise 2
#
# Complete this segment demonstrating tool calling capabilites. Have the model sucessfully use your tool.
#

import typing as t

class Say(rg.Tool):
  @property
  def name(self) -> str:
    return "say"

  @property
  def description(self) -> str:
    return "Say a phrase to the user"

  def say(self, message: t.Annotated[str, "Message the show"]) -> str:
    print(f'\n\n model says: {message}\n\n')
    

say = Say()
chat = generator.chat([{'role': 'user', 'content': 'Say Hello using the tool'}]).using(say).run()


In [ ]:
[print(m) for m in chat.all]

#  OvertheWire
In this workshop we're going to explore building an agent with [rigging]((https://github.com/dreadnode/marque)) and [marque]((https://github.com/dreadnode/marque)) to solve CTF challenge hosting on [OverTheWire](https://overthewire.org/wargames/). OvertheWire is an online platform that offers a series of wargames designed to help users learn and practice various cybersecurity and system administration concepts. The wargames are played in a secure hosted environment and consist of a set of challenges that gradually increase in difficulty. Each wargame typically focuses on a specific area of experitse, in this lab we're going to look at the [Bandit](https://overthewire.org/wargames/bandit/) series of challenges.

# Summary
The Bandit wargame is an introductory game designed for absolute beginners to teach them the basics needed to play other wargames. The game is organized into levels, starting from [Level 0](https://overthewire.org/wargames/bandit/bandit0.html), and players must complete each level to progress to the next. Information on how to start each level is provided on the respective level's page on the OverTheWire website.

Please go and read the prompt for the first challenge!

In [6]:
import requests
import re

BASE_DOCS_URL = 'https://overthewire.org/wargames/bandit'

def get_bandit_level_details(level: int) -> str:
    search = r'Level Goal</h2>(.+)<h2'
    response = requests.get(f'{BASE_DOCS_URL}/bandit{level}.html')
    response.raise_for_status()
    goal: str = re.findall(search, response.text, re.DOTALL)[0]
    return goal.replace('<p>', '').replace('</p>', '').strip()

In [7]:
import typing as t
import rigging as rg
import paramiko

SSH_HOST = 'bandit.labs.overthewire.org'
SSH_PORT = 2220

class ChallengeTool(rg.Tool):
    @property
    def name(self) -> str:
        return "challenge"

    @property
    def description(self) -> str:
        return "Provides access to the challenge environment"

    def __init__(self, host: str, port: int, username: str, password: str):
        self.host = host
        self.port = port
        self.username = username
        self.password = password
        self.client: paramiko.SSHClient | None = None

    def _connect(self) -> None:
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy)
        self.client.connect(self.host, self.port, self.username, self.password)

    def execute(self, command: t.Annotated[str, "The plaintext command with no variables"]) -> str:
        "Execute a bash command on the host"
        if self.client is None:
            self._connect()

        print(f'Executing {command}')
        _, stdout_channel, stderr_channel = self.client.exec_command(command, timeout=5)
        stdout = stdout_channel.read().decode()
        stderr = stderr_channel.read().decode()

        output = stderr + '\n' + stdout

        if not output.strip():
            output = '[No output was provided or a timeout was hit]'

        print(f'Output: {output}')

        return output

    def get_banner(self) -> str:
        "Get the banner information from the host"
        return self.execute('cat /etc/motd')


In [8]:
from marque import Flow, MemoryStorage
from rigging.generator import Generator
from rigging.model import Answer, Thinking
import marque.logging
import rigging.logging

marque.logging.configure_logging("trace")
rigging.logging.configure_logging("debug")

SYSTEM_PROMPT = """\
You are a senior security expert.
"""

def get_prompt(level: int, **resources: str) -> str:
    goal = get_bandit_level_details(level)
    resource_str = "\n".join([f"<{k}>\n{v}\n</{k}>" for k, v in resources.items()])
    return f"""\
You have been provided access to an SSH session on a host for a CTF challenge. Through this SSH session you will work to recover the password for the next account level. Here is the current challenge description:

<challenge level="{level}>
{goal}
</challenge>

<resources>
{resource_str}
</resources>

When you believe you have recovered the password to the next level, place the text within {Answer.xml_tags()} tags. Before you begin this task, document your understanding of the task, potential mistakes you might make, and good strategies for accomplishing the task between {Thinking.xml_tags()} tags. Use the tools you have available to you to execute one command a time and retrieve results.

Be persistent and think hard about why steps might be failing. Good luck!
"""


def solve(flow: Flow) -> None:
    level = flow.get(int, 'level')
    next_level = level + 1
    password = flow.get(str, "password")
    username = f'bandit{level}'

    generator = flow.get(Generator)

    tool = ChallengeTool(SSH_HOST, SSH_PORT, username, password)

    flow.log('Authenticating ...')
    tool._connect()

    flow.log('Asking the model ...')

    pending = generator.chat(
        [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": get_prompt(next_level)},
        ]
    )

    chat = pending.until_parsed_as(Answer, max_rounds=100).using(tool).run()

    answer = chat.last.parse(Answer)
    next_password = answer.content.strip().strip('.-|,')
    flow.success(f'Level {next_level} password: {next_password}')

    flow.success('Pushing next solve step')
    flow.push(solve, level=next_level, password=next_password)

flow = Flow("ctf", MemoryStorage()).fail_fast()

generator = rg.get_generator(connection_string)

flow.put(
    level=0,
    password="bandit0",
    generator=generator
).push(solve)()

18:10:48.943 | [+] 
18:10:48.944 | [+] Starting flow 'ctf' / 'vivacious-ibis'
18:10:48.945 | [+] 
18:10:48.946 | [=] > Step 'solve' (0:0)
18:10:50 - LiteLLM:INFO: 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1/ \
-d '{'model': 'mistral-large-latest', 'messages': [{'role': 'system', 'content': 'You are a senior security expert.\n\n\nIn this environment you have access to a set of tools you can use to improve your responses.\n\nTool call format:\n<tool-calls>\n   <tool-call tool="$TOOL_A" function="$FUNCTION_A">\n      <parameter name="$PARAMETER_NAME" />\n   </tool-call>\n   <tool-call tool="$TOOL_B" function="$FUNCTION_B">\n      <parameter name="$PARAMETER_NAME" />\n   </tool-call>\n</tool-calls>\n\nAvailable tools:\n<tool-description-list>\n   <tool-description name="challenge" description="Provides access to the challenge environment">\n      <functions>\n         <tool-function name="execute" description="Execute a bash command on the host">\n           

KeyboardInterrupt: 